<a href="https://colab.research.google.com/github/lalaxd09/Procesamiento-de-Lenguaje-Natural/blob/main/Chat_Bot_PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tflearn
!pip install tensorflow==1.15

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 34.3 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=03a98a86476665e6718a7372afb1ac8786186646a53f69f8de44bfd143dd6541
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 412.3 MB 26 kB/s 
     |████████████████████████████████| 50 kB 7.6 MB/s 
     |████████████████████████████████| 3.8 MB 56.4 MB/s 
     |████████████████████████████████| 503 kB 62.5 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=e7841e667de892dd87dce2e57cd2b1c13cc71b419fb648f0aa613f68682cc361
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823d

In [2]:
!pip install nltk
!pip install json
!pip install pickle
!pip install warnings

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)
ERROR: No matching distribution found for warnings


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()
import tflearn
import random

#Usde to for Contextualisation and Other NLP Tasks.
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
print("Processing the Intents.....")
with open('/content/drive/My Drive/tristongo.json') as json_data:
  intents = json.load(json_data)

Processing the Intents.....


In [6]:
words = []
classes = []
documents = []
ignore_words = ['?']
print("Looping through the Intents to Convert them to words, classes, documents and ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tOKENIZA CADA PALABRA
        w = nltk.word_tokenize(pattern)
        # aGREGAMOS A NUESTRA LISTA DE PAALBRAS
        words.extend(w)
        # AGREAGAMOS DOCUMENTOS A NUESTRO ETIQUETAS
        documents.append((w, intent['tag']))
        # AGREAGAMOS A NUESTRA CLASE DE LISTA
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Looping through the Intents to Convert them to words, classes, documents and ignore_words.......


In [7]:
print("Stemming, Lowering and Removing Duplicates.......")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# SE REMUEVEN LOS DUPLICADO
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Stemming, Lowering and Removing Duplicates.......
61 documents
23 classes ['Acuario', 'Aries', 'Capricornio', 'Escorpio', 'Gemenis', 'Leo', 'Libra', 'Psicis', 'Sagitario', 'Tauro', 'Virgo', 'cancer', 'canciones_favs', 'drogras', 'goodbye', 'greeting', 'gustos', 'horoscopos', 'musica', 'respuesta_Tristongo', 'sentimiento', 'thanks', 'tristongo']
83 unique stemmed words [',', 'acerc', 'acuario', 'adio', 'adiooo', 'agradecido', 'ah', 'algo', 'algun', 'alguy', 'anda', 'ar', 'arrib', 'ary', 'as', 'ayud', 'buen', 'bye', 'canción', 'cant', 'capricornio', 'con', 'conoc', 'de', 'droga', 'día', 'el', 'encuentra', 'escorpio', 'escuch', 'esta', 'fav', 'favorita', 'gem', 'gemin', 'gracia', 'gust', 'hac', 'hay', 'haz', 'hol', 'horoscopo', 'horospoco', 'la', 'leo', 'libr', 'los', 'luego', 'me', 'metet', 'morir', 'music', 'música', 'nos', 'op', 'oído', 'por', 'psic', 'pued', 'que', 'quiero', 'recomend', 'sab', 'sagitario', 'sient', 'soy', 'suicidac', 'tal', 'tauro', 'te', 'thx', 'tipo', 'todo', 'trist

In [8]:
print("Creating the Data for our Model.....")
training = []
output = []
print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)

print("Creating Traning Set, Bag of Words for our Model....")
for doc in documents:
    # incializa bolsa de palabra
    bag = []
    # tokeniza las palabras para el patron
    pattern_words = doc[0]
  
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # crea nuestra bolsa de palabras en un arreglo
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # salida es un  '0' para cada etiqueta y '1' para la actual
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Creating the Data for our Model.....
Creating an List (Empty) for Output.....
Creating Traning Set, Bag of Words for our Model....


In [9]:
print("Shuffling Randomly and Converting into Numpy Array for Faster Processing......")
random.shuffle(training)
training = np.array(training)

print("Creating Train and Test Lists.....")
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Building Neural Network for Out Chatbot to be Contextual....")
print("Resetting graph data....")


Shuffling Randomly and Converting into Numpy Array for Faster Processing......
Creating Train and Test Lists.....
Building Neural Network for Out Chatbot to be Contextual....
Resetting graph data....


In [10]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
print("Trainingp....")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Trainingp....


In [11]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
print("Training the Model.......")
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
print("Saving the Model.......")
model.save('model.tflearn')

Training Step: 7999  | total loss: 0.08574 | time: 0.036s
| Adam | epoch: 1000 | loss: 0.08574 - acc: 0.9716 -- iter: 56/61
Training Step: 8000  | total loss: 0.07766 | time: 0.041s
| Adam | epoch: 1000 | loss: 0.07766 - acc: 0.9744 -- iter: 61/61
--
Saving the Model.......
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [12]:
print("Pickle is also Saved..........")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [13]:
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('/content/drive/My Drive/tristongo.json') as json_data:
    intents = json.load(json_data)
    
print("Loading the Model......")
# carga nuestro modelo
model.load('./model.tflearn')

Loading Pickle.....
Loading the Model......
INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [14]:
def clean_up_sentence(sentence):
    # Lo tokeniza o lo divide en las partes constituyentes de Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    #Stemming significa encontrar la raíz de la palabra.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

#Regresa la bolsa de valores
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    # Predicción o para obtener la posibilidad o probabilidad del modelo
    results = model.predict([bow(sentence, words)])[0]
    # exluye los valores con umbral
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # La clasificación se realiza porque la respuesta de mayor confianza es lo primero.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> intenta y la probabilidad
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    
    if results:
        
        while results:
            for i in intents['intents']:
                
                if i['tag'] == results[0][0]:
                   
                    return print(random.choice(i['responses']))

            results.pop(0)

ERROR_THRESHOLD = 0.25


In [ ]:
while True:
    input_data = input("You- ")
    answer = response(input_data)
    answer